In [1]:
import requests
import json
import getpass
from sqlalchemy import create_engine
import pandas as pd
from snowflake.sqlalchemy import URL

In [2]:
import sqlalchemy
import snowflake

In [3]:
#!pip feeze

In [4]:
# environment is either 'nonprod' or 'prod'
environment = 'prod'
seven_letter = 'staajam'
sf_role = 'surfacetrans_consumer'
sf_warehouse = 'SURFACETRANS_WAREHOUSE'

base_url = f'https://vault-{ environment }.centralus.chrazure.cloud'

In [5]:
def get_client_token(username, password):
    payload = {
        'password': password
    }

    response = requests.post(f'{ base_url }/v1/auth/okta/login/{ seven_letter.lower() }', data=payload, verify=False)

    client_token = response.json()['auth']['client_token']

    return client_token

In [6]:
def get_leased_password(username, client_token):
    CONN = 'snowflake'
    RL = username.lower()

    if environment == 'prod':
        DB = 'prod'
    else:
        DB = 'dev'

    headers = {
        'X-Vault-Token': client_token
    }

    response = requests.get(f'{ base_url }/v1/database/static-creds/{ CONN }_{ DB }_{ RL }', headers=headers, verify=False)
    print(response.json())
    leased_password = response.json()['data']['password']

    return leased_password

In [7]:
#!pip freeze

In [8]:
def get_sqlalchemy_engine(username, leased_password):
    USER = f'{ username.upper() }@CHROBINSON.COM'
    PASSWORD = leased_password

    if environment == 'prod':
        ACCOUNT = 'prod_chrobinson.east-us-2.azure'
    else:
        ACCOUNT = 'dev_chrobinson.east-us-2.azure'
        
    url = URL(
        user=USER,
        password=PASSWORD,
        account=ACCOUNT,
        role=sf_role,
        warehouse=sf_warehouse
    )
    
    engine = create_engine(url)

    return engine

In [9]:
client_token = get_client_token(seven_letter, getpass.getpass(prompt='CHR password... '))
leased_password = get_leased_password(seven_letter, client_token)
engine = get_sqlalchemy_engine(seven_letter, leased_password)
conn = engine.connect()

/home/chrobinson.com/staajam/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vault-prod.centralus.chrazure.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/chrobinson.com/staajam/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vault-prod.centralus.chrazure.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'request_id': '87ad9927-44dd-30fd-813c-77aa4f9d5ab4', 'lease_id': '', 'renewable': False, 'lease_duration': 0, 'data': {'last_vault_rotation': '2021-10-03T20:01:28.241057738Z', 'password': 'm0gJknZUliG-EQqzTRD3', 'rotation_period': 86400, 'ttl': 4462, 'username': 'STAAJAM@CHROBINSON.COM'}, 'wrap_info': None, 'warnings': None, 'auth': None}


In [10]:
sql = 'select \
    CAST(ar.CHR_ACCOUNTREQUESTID as CHAR(32)) "Id", \
    ar.CHR_ACCOUNTNAME "Name", \
    ar.CHR_ADDRESS1STREET1 "Street", \
    ar.CHR_ADDRESS1POSTALCODE "Postal Code", \
    ar.CHR_ADDRESS1CITY "City", \
    ar.CHR_EMAILADDRESS "Email", \
    ar.CHR_WEBSITEURL "Website", \
    ar.CHR_PHONENUMBER "Phone",\
    ar.CHR_DUNSNUMBER "Duns", \
    sc.CHR_NAME "Country", \
    ar.CHR_REQUESTTYPECODE "Type", \
    ar.CHR_REQUESTSTATUSCODE "Status" \
    from CDC_ATMOSPHERE.TMC_ALL.DBO_CHR_ACCOUNTREQUEST ar \
    inner join CDC_ATMOSPHERE.TMC_ALL.DBO_CHR_STATECOUNTRY sc on ar.CHR_ADDRESS1COUNTRYID = sc.ID \
    where ar.CREATEDON between DATEADD(Day ,-7, current_date) and DATEADD(Day ,+1, current_date) \
    group by \
    CAST(ar.CHR_ACCOUNTREQUESTID as CHAR(32)), \
    ar.CHR_ACCOUNTNAME, \
    ar.CHR_ADDRESS1STREET1, \
    ar.CHR_ADDRESS1POSTALCODE, \
    ar.CHR_ADDRESS1CITY, \
    ar.CHR_EMAILADDRESS, \
    ar.CHR_WEBSITEURL, \
    ar.CHR_PHONENUMBER, \
    ar.CHR_DUNSNUMBER, \
    sc.CHR_NAME, \
    ar.CHR_REQUESTTYPECODE, \
    ar.CHR_REQUESTSTATUSCODE;'

In [14]:
df1=pd.read_sql(sql, conn)
print(len(df1))

338


In [13]:
conn

In [12]:
df1

,Id,Name,Street,Postal Code,City,Email,Website,Phone,Duns,Country,Type,Status
0,5C625D9920E111ECB6E6000D3A55711C,"Jia Yi Trading (JY) Co., Ltd.","1 F., No. 160, Baoyi Rd., Houhu Vil., East Dis...",60079,Jiaya,None,None,05-2322266,None,Taiwan (Province of China),126770000,126770001
1,9E57DFAA21CE11ECB6E6000D3A1675A8,"Alchamp international Co., Ltd.","1F, No. 21, Lane 255, Zhongxing St., Xingnong Vil",72143,Tainan,None,None,None,657748184,Taiwan (Province of China),126770000,126770001
2,68ED14D7212711ECB6E6000D3A1675A8,RASPBERRY PI (TRADING) Ltd.,"Maurice Wilkes Building, St. John's Innovation...",CB4 0DS,CAMBRIDGE,sarah.roth@raspberrypi.com,http://www.raspberrypi.org/,+44-1223322633,218577959,United Kingdom of Great Britain and Northern I...,126770000,126770001
3,8FAC4474203911ECB6E6000D3A13F018,BENTLEY DESIGNS (UK) Ltd.,"Unit A1 Symmetry Park, Morrell Way",OX26 6GF,BICESTER,rif@bentleydesigns.com,www.bentleydesigns.com,+44-2088337500,397067331,United Kingdom of Great Britain and Northern I...,126770000,126770001
4,51DC6EA9210711ECB6E6000D3A167AE9,CHIPITA UK LIMITED,"Shakespeare House, 42 Newmarket Road",CB5 8EP,Cambridge,None,None,None,None,United Kingdom of Great Britain and Northern I...,126770000,126770002
...,...,...,...,...,...,...,...,...,...,...,...,...
333,670AA33F220511ECB6E6000D3A1276A5,PROGEKTA S.R.L.S.,Via Molise 9,20098,San Giuliano Milanese,None,None,None,440894060,Italy,126770000,126770001
334,D765BA06229111ECB6E6000D3A12E50E,WITTUR ASANSOR SAN VE TIC AS,"Dudullu Osb, No:13 2. CaddeDudullu Orgazine Sa...",333921,Istanbul (Anatolia),None,http://www.wittur.com/,+90-2164155567,644652273,Turkey,126770000,126770001
335,D6BA35B5228611ECB6E6000D3A557DE4,Versalis Kimya Ticaret Ltd. Sti.,Büyükdere Cad. Noramin İşmerk. No:237 K:4 Ofis...,34000,Istanbul,None,None,None,None,Turkey,126770000,126770002
336,138364C921DA11ECB6E6000D3A557DE4,SAFIC ALCAN KIMYA SAN VE TIC AS,"D Blok K:4 D:408, No:8 Ikitelli Osb Ziya Gokal...",34490,Istanbul (Europe),None,http://www.safic-alcan.com.tr/,+90-2125496950,356044797,Turkey,126770000,126770001
